# ANSWER KEY: Debug Drill 07 - Wrong K

**Bug:** Colleague used K=20 clusters ("more = better"), resulting in redundant, non-actionable segments.

**Key Lesson:** Use the Elbow Method to find optimal K. Segments should be distinct and actionable.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

df = pd.read_csv('https://raw.githubusercontent.com/189investmentai/ml-foundations-interactive/main/data/streamcart_customers.csv')

cluster_features = ['tenure_months', 'logins_last_30d', 'orders_last_30d', 'support_tickets_last_30d']
X = df[cluster_features].fillna(0)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

## The Bug (Colleague's Code)

In [ ]:
# ===== BUGGY CODE =====
k_buggy = 20
kmeans_buggy = KMeans(n_clusters=k_buggy, random_state=42, n_init=10)
df['cluster_buggy'] = kmeans_buggy.fit_predict(X_scaled)

print(f"Colleague's approach: K={k_buggy} clusters")
print(f"\nCluster sizes:")
print(df['cluster_buggy'].value_counts().sort_index())

In [ ]:
# Show redundant clusters
profiles_buggy = df.groupby('cluster_buggy')[cluster_features].mean().round(1)
print("\nLook at similar clusters:")
print(profiles_buggy.head(10))
print("\nMany clusters have nearly identical profiles!")

## Why This Is Wrong

**Problems with K=20:**
1. Many clusters have identical or near-identical profiles
2. Marketing can't create 20 different campaigns
3. Some clusters are too small to be actionable
4. Segment distinctions become meaningless

**The "more is better" fallacy:**
- More clusters ≠ more insight
- Clusters should be DISTINCT and ACTIONABLE

## The Fix: Elbow Method

In [ ]:
# ===== FIXED CODE =====
# Step 1: Use Elbow Method to find optimal K

inertias = []
k_range = range(2, 12)

for k in k_range:
    km = KMeans(n_clusters=k, random_state=42, n_init=10)
    km.fit(X_scaled)
    inertias.append(km.inertia_)

plt.figure(figsize=(10, 5))
plt.plot(k_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters (K)', fontsize=12)
plt.ylabel('Inertia (within-cluster sum of squares)', fontsize=12)
plt.title('Elbow Method - Find the Bend', fontsize=14)
plt.axvline(x=4, color='r', linestyle='--', label='Elbow at K=4')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("The 'elbow' appears around K=4.")
print("Adding more clusters beyond this provides diminishing returns.")

In [ ]:
# Step 2: Apply optimal K
k_optimal = 4

kmeans_fixed = KMeans(n_clusters=k_optimal, random_state=42, n_init=10)
df['cluster'] = kmeans_fixed.fit_predict(X_scaled)

print(f"\n=== {k_optimal} CLUSTERS ===")
print(f"\nCluster sizes:")
print(df['cluster'].value_counts().sort_index())

In [ ]:
# Step 3: Profile the clusters
profiles = df.groupby('cluster')[cluster_features + ['churn_30d']].mean().round(2)
profiles.columns = ['Tenure', 'Logins', 'Orders', 'Tickets', 'Churn Rate']
print("\nCluster Profiles:")
print(profiles)

In [ ]:
# Step 4: Name the clusters based on profiles
# Analyze each cluster and assign business-meaningful names

cluster_analysis = profiles.copy()

def name_cluster(row):
    tenure = row['Tenure']
    logins = row['Logins']
    churn = row['Churn Rate']
    
    if churn > 0.15:
        risk = "High-Risk"
    elif churn < 0.05:
        risk = "Loyal"
    else:
        risk = "Moderate-Risk"
    
    if tenure > 12:
        tenure_label = "Veterans"
    elif tenure < 4:
        tenure_label = "New"
    else:
        tenure_label = "Established"
    
    return f"{risk} {tenure_label}"

cluster_names = {i: name_cluster(profiles.loc[i]) for i in range(k_optimal)}

print("\nCluster Names:")
for c, name in cluster_names.items():
    churn = profiles.loc[c, 'Churn Rate']
    size = (df['cluster'] == c).sum()
    print(f"  Cluster {c}: {name}")
    print(f"    Size: {size:,} customers")
    print(f"    Churn rate: {churn:.1%}")

In [ ]:
# Validate: Clusters should have different churn rates
churn_rates = df.groupby('cluster')['churn_30d'].mean()
churn_spread = churn_rates.max() - churn_rates.min()

print(f"\nChurn rate spread: {churn_spread:.1%}")
print("(Higher spread = more distinct segments)")

assert churn_spread > 0.05, "Clusters should have meaningfully different churn rates"
assert k_optimal < 10, "K should be manageable for business"
print("\nPASS: Clusters are distinct and actionable!")

## Marketing Recommendations by Segment

| Segment | Churn Risk | Recommended Action |
|---------|------------|--------------------|
| Loyal Veterans | Low | Referral program, upsell |
| High-Risk New | High | Onboarding support, early engagement |
| Moderate-Risk Established | Medium | Retention offer, check-in |
| High-Risk Veterans | High | Win-back campaign, VIP support |

## Completed Postmortem

### Root cause:
- Assumed "more clusters = better granularity" without validating cluster quality
- Didn't use Elbow Method to find natural groupings in the data

### How we detected it:
- Marketing complained that clusters looked identical
- Many clusters had similar profiles and overlapping characteristics

### Prevention for next time:
- Always use Elbow Method (or silhouette score) to find optimal K
- Validate clusters are distinct: different churn rates, different profiles
- Ask: "Can marketing actually use 20 different segments?" (usually no)